# Twitch data analysis

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize

In [2]:
completelist = [os.path.join(path, name) for path, subdirs, files in os.walk("./files_stream/") for name in files]
print("Number of files:", len(completelist))
print(*completelist[:10], sep = "\n")

Number of files: 45
./files_stream/2022-04-15_13-32.txt
./files_stream/2022-04-15_14-50.txt
./files_stream/2022-04-15_14-56.txt
./files_stream/2022-04-15_21-34.txt
./files_stream/2022-04-15_21-49.txt
./files_stream/2022-04-15_21-50.txt
./files_stream/2022-04-15_21-51.txt
./files_stream/2022-04-15_22-00.txt
./files_stream/2022-04-15_22-09.txt
./files_stream/2022-04-15_22-11.txt


### Sampling frequency

In [21]:
flist= []
#set up sample frequency: 1=5min, 2=10min, 3=15min ecc.
freq= 2
flist= completelist[::freq]

In [5]:
for i in range(0, len(flist)):
    print(flist[i])

./files_stream/2022-04-15_13-32.txt
./files_stream/2022-04-15_14-56.txt
./files_stream/2022-04-15_21-49.txt
./files_stream/2022-04-15_21-51.txt
./files_stream/2022-04-15_22-09.txt
./files_stream/2022-04-15_22-14.txt
./files_stream/2022-04-15_22-31.txt
./files_stream/2022-04-15_22-47.txt
./files_stream/2022-04-15_22-57.txt
./files_stream/2022-04-27_12-00.txt
./files_stream/2022-04-27_12-05.txt
./files_stream/2022-04-27_12-09.txt
./files_stream/2022-04-27_12-18.txt
./files_stream/2022-04-27_12-29.txt
./files_stream/2022-04-27_12-37.txt
./files_stream/2022-04-28_16-52.txt
./files_stream/2022-04-28_17-05.txt
./files_stream/2022-04-28_17-42.txt
./files_stream/2022-04-28_17-45.txt
./files_stream/2022-05-01_17-39.txt
./files_stream/2022-05-01_17-45.txt
./files_stream/2022-05-01_18-22.json
./files_stream/2022-05-05_14-35.json


### Import selected files

### Con dataframe

In [6]:
df= pd.DataFrame( columns= ['spect', 'game_name', 'viewer_count', 'timestamp'])

for file in flist:
    
    #extract timestamp
    start= './files_stream/'
    end= '.json'
    time= file[len(start): - len(end)]
    datetime_obj= datetime.strptime(time, '%Y-%m-%d_%H-%M')
    
    with open(file) as f:
        data= f.readlines()
        #elimina parentesi quadra
        data= data[0]
        
        #prende in formato json
        file_j= json.loads(data)
        
        #converti json in dataframe
        for i in file_j:
            name= str(i)
            streamer= json_normalize(file_j[name])
            streamer['timestamp']= datetime_obj
            streamer['streamer']= name
            df= df.append(streamer, ignore_index= True)

In [7]:
df.shape

(49072, 5)

In [8]:
df

,spect,game_name,viewer_count,timestamp,streamer
0,"[0_0ensar0_0, 0marr_, 11ed0, 12345aar, 16lxnes...",Fortnite,2375,2022-04-15 13:03:00,Xiuder_
1,"[00rickk, 17davinc17, 1_x_2_over, 1uca74, 2001...",Just Chatting,2299,2022-04-15 13:03:00,Ivan_Grieco
2,"[00peppe_, 0zy_mandias_50, 3453373073, 34frtg4...",Just Chatting,1156,2022-04-15 13:03:00,NanniTwitch
3,"[0mar_01, 11minifish, 20th_century_boy90, 2saf...",Just Chatting,1124,2022-04-15 13:03:00,voghelita
4,"[actionshin, ago1927, aiacee, akuma_nakami, al...",Just Chatting,813,2022-04-15 13:03:00,Everyeyeit
...,...,...,...,...,...
49067,"[casinothanks, lurxx, own3d, tinarif]",Call of Duty: Warzone,0,2022-05-05 14:35:00,zigzac_tv
49068,"[0ax2, dankingaround, edellyna, lurxx, santacl...",Corpse Party,0,2022-05-05 14:35:00,miss_galaco
49069,"[0ax2, aliengathering, kattynah, lurxx, sixfla...",Call of Duty: Warzone,0,2022-05-05 14:35:00,Bersik_
49070,"[lurxx, mogulmail, own3d]",League of Legends,0,2022-05-05 14:35:00,davesanna98


In [9]:
df['game_name'].value_counts()

Fortnite                                 5087
Call of Duty: Warzone                    3797
Just Chatting                            3109
Call Of Duty: Modern Warfare             2430
Grand Theft Auto V                       2236
                                         ... 
Platforms                                   1
Banished                                    1
Uncharted: Drake's Fortune Remastered       1
Barony                                      1
FIFA Mobile                                 1
Name: game_name, Length: 1072, dtype: int64

### Con dizionari

In [14]:
data_dict = {}

for file in flist:
    
    #extract timestamp
    start = './files_stream/'
    end = '.json'
    time = file[len(start): - len(end)]
    time_obj = datetime.strptime(time, '%Y-%m-%d_%H-%M')
    
    with open(file, 'r') as f:
        data_dict[time_obj] = json.load(f)

print("Number of keys (time intervals):", len(data_dict.keys()))

Number of keys (time intervals): 26


Verifichiamo la differenza tra viewer e spect

In [15]:
#aggiungere ciclo for su tutti gli istanti

for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    
    print(f"{streamer}\n\
            Viewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
            Spect Count: {len(data_dict[time_obj][streamer]['spect'])}\
            Difference: {data_dict[time_obj][streamer]['diff_count']}\n")

Moonryde
            Viewer Count: 3472            Spect Count: 2530            Difference: 942

ilMasseo
            Viewer Count: 3266            Spect Count: 2809            Difference: 457

Xiuder_
            Viewer Count: 2635            Spect Count: 1769            Difference: 866

Ivan_Grieco
            Viewer Count: 2592            Spect Count: 1715            Difference: 877

Juventibus
            Viewer Count: 1889            Spect Count: 1340            Difference: 549

Justees
            Viewer Count: 1515            Spect Count: 1297            Difference: 218

NanniTwitch
            Viewer Count: 1229            Spect Count: 1086            Difference: 143

fragolaqt
            Viewer Count: 1080            Spect Count: 915            Difference: 165

Everyeyeit
            Viewer Count: 1072            Spect Count: 80            Difference: 992

Paoloidolo
            Viewer Count: 947            Spect Count: 820            Difference: 127

MarcoMerrino
           

Le differenze sono dovute a:

- bot rimossi dal viewer count
- persone non connesse in chat
- aggiornamento dati diverso tra i due

### Remove bot

In [16]:
bot_list= pd.read_csv('bot_list.csv', header= None)
bot_list= bot_list[0].tolist()

In [17]:
def remove_bot(a):
    clean_list = []
    for chatter in a:
        if chatter not in bot_list:
            clean_list.append(chatter)
    return clean_list

In [18]:
data_dict[time_obj][streamer]['spect']

['0ax2', 'aliengathering', 'dankingaround', 'gowithhim', 'lylituf']

In [19]:
len(data_dict[time_obj][streamer]['spect'])

5

In [20]:
data_dict[time_obj][streamer]['spect']= remove_bot(data_dict[time_obj][streamer]['spect'])
len(data_dict[time_obj][streamer]['spect'])

0

### Obtain streamer list and count

In [ ]:
streamer_repeated = []

for istante in data_dict.keys():
    streamer_list.extend(list(data_dict[istante].keys()))

len(streamer_list)

In [ ]:
from collections import Counter

streamer_list = dict(Counter(streamer_list))
len(a.keys())